In [1]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 725.9 kB/s eta 0:00:00


In [2]:
!pip install python-dotenv

In [4]:
!pip install langchain openai nbformat faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [12]:
from google.colab import drive
drive.mount('/content/drive')

import os
import openai
from dotenv import load_dotenv

env_path = '/content/drive/MyDrive/DL_HW11/config.env'

load_dotenv(env_path)

openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print("API Key loaded successfully!")
    print("API Key ends with:", openai_api_key[-4:])
    openai.api_key = openai_api_key
else:
    print("Failed to load API Key.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
API Key loaded successfully!
API Key ends with: BNQZ


# PART 1: code understanding model that handles both .py an .ipynb file

In [18]:
import os
import nbformat

def extract_py_content(file_path):
    """Read Python file and return its content."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def extract_ipynb_content(file_path):
    """Read Jupyter notebook file and return code and markdown cells."""
    with open(file_path, 'r', encoding='utf-8') as file:
        nb = nbformat.read(file, as_version=4)
        code = ''
        markdown = ''
        for cell in nb.cells:
            if cell.cell_type == 'code':
                code += cell.source + '\n'
            elif cell.cell_type == 'markdown':
                markdown += cell.source + '\n'
        return code, markdown


In [19]:
import openai
from transformers import AutoTokenizer, AutoModel
import numpy as np
import faiss

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(1).detach().numpy()


dimension = 384
index = faiss.IndexFlatL2(dimension)

def add_to_index(texts):
    embeddings = np.vstack([get_embedding(text) for text in texts])
    index.add(embeddings)


documents = []
for filename in os.listdir('/content/drive/MyDrive/DL_HW12/code_input'):
    if filename.endswith('.py'):
        content = extract_py_content(os.path.join('/content/drive/MyDrive/DL_HW12/code_input', filename))
        print(content)
        documents.append(content)
        add_to_index([content])
    elif filename.endswith('.ipynb'):
        code, markdown = extract_ipynb_content(os.path.join('/content/drive/MyDrive/DL_HW12/code_input', filename))
        print(code)
        documents.append(code)
        add_to_index([code])



{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"provenance":[],"authorship_tag":"ABX9TyN/ZpINDQsqmjm20x4CHpEY"},"kernelspec":{"name":"python3","display_name":"Python 3"},"language_info":{"name":"python"}},"cells":[{"cell_type":"code","execution_count":null,"metadata":{"id":"DTSq4H0vaUNc"},"outputs":[],"source":["def Fibonacci(n):\n","\n","\t# Check if input is 0 then it will\n","\t# print incorrect input\n","\tif n < 0:\n","\t\tprint(\"Incorrect input\")\n","\n","\t# Check if n is 0\n","\t# then it will return 0\n","\telif n == 0:\n","\t\treturn 0\n","\n","\t# Check if n is 1,2\n","\t# it will return 1\n","\telif n == 1 or n == 2:\n","\t\treturn 1\n","\n","\telse:\n","\t\treturn Fibonacci(n-1) + Fibonacci(n-2)\n","\n","\n","# Driver Program\n","print(Fibonacci(9))\n"]}]}


In [20]:
def simple_retrieve(query, k=5):
    query_embedding = get_embedding(query)
    _, indices = index.search(query_embedding, k)
    return [documents[i] for i in indices[0]]


In [30]:
def generate_answer(question):
    # Retrieve relevant contexts
    contexts = simple_retrieve(question)
    context_string = "\n".join(contexts)  # Combine contexts into a single string

    # Format the messages for the chat model
    messages = [
        {"role": "system", "content": "You are a helpful code understanding assistant. answer the question only based on the context and reason."},
        {"role": "user", "content": f"{context_string}\nQuestion: {question}"}
    ]

    # Query OpenAI API with context and question using the chat API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=150,
        temperature=0.5,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=["\n"]
    )

    return response['choices'][0]['message']['content'].strip()


In [31]:
question = "What does the fibonacci function do?"
response = generate_answer(question)
print("Response:", response)


Response: The Fibonacci function calculates the nth Fibonacci number recursively using the formula F(n) = F(n-1) + F(n-2), where F(0) = 0, F(1) = 1, and F(2) = 1.


In [32]:
question = "which number does the driver program pass to the  fibonacci function?"
response = generate_answer(question)
print("Response:", response)


Response: The driver program passes the number 9 to the Fibonacci function.


In [33]:
question = "what happens if the 1 is passed to the function?"
response = generate_answer(question)
print("Response:", response)


Response: If 1 is passed to the function, it will return 1 because the function is designed to return 1 if the input is equal to 1.


In [34]:
question = "explain fibonacci number?"
response = generate_answer(question)
print("Response:", response)


Response: Fibonacci numbers are a sequence of numbers in which each number is the sum of the two preceding ones, usually starting with 0 and 1. The sequence goes like this: 0, 1, 1, 2, 3, 5, 8, 13, and so on. In the provided code snippet, there is a Python function defined to calculate the nth Fibonacci number recursively.


In [35]:
question = "help me understand this code?"
response = generate_answer(question)
print("Response:", response)


Response: This code defines a function called Fibonacci that calculates the Fibonacci sequence recursively. It first checks if the input is less than 0 and prints "Incorrect input" if it is. Then it handles the base cases where if n is 0, it returns 0, and if n is 1 or 2, it returns 1. For any other value of n, it recursively calls the Fibonacci function with n-1 and n-2 and returns the sum of the results. Finally, it prints the result of calling Fibonacci(9).


# PART 2:

In [ ]:
# AWS Case study and blog - text files downloaded from kaggle is used for this module.

In [53]:
import os
import openai
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel

# Initialize tokenizer and model for embeddings
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()




In [54]:


dimension = 384
index = faiss.IndexFlatL2(dimension)
documents = []
document_ids = []


def load_documents(folder_path):
    doc_id = 0
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                documents.append(text)
                document_ids.append(doc_id)
                embedding = get_embedding(text)

                if embedding.ndim == 1:
                    embedding = embedding.reshape(1, -1)
                elif embedding.ndim == 2:
                    pass
                else:
                    raise ValueError("Embedding dimension mismatch: expected 1 or 2 dimensions, got {}".format(embedding.ndim))
                index.add(embedding)
                doc_id += 1


In [55]:

class DocumentRetrieval:
    def __init__(self, index, documents, document_ids):
        self.index = index
        self.documents = documents
        self.document_ids = document_ids

    def retrieve(self, query, k=5):
        query_embedding = get_embedding(query)

        if query_embedding.ndim == 1:
            query_embedding = query_embedding.reshape(1, -1)
        elif query_embedding.ndim == 2:
            pass
        else:
            raise ValueError("Query embedding dimension mismatch: expected 1 or 2 dimensions, got {}".format(query_embedding.ndim))
        # Perform the search
        _, indices = self.index.search(query_embedding, k)
        return [self.documents[i] for i in indices[0]]


retriever = DocumentRetrieval(index, documents, document_ids)
load_documents("/content/drive/MyDrive/DL_HW12/part2_input")



In [59]:

conversation_history = []

def chatbot(question):
    # Retrieve relevant documents
    retrieved_docs = retriever.retrieve(question)
    context = " ".join(retrieved_docs)
    prompt = f"{context}\n\n{question}"


    messages = [
        {"role": "system", "content": "You are a helpful assistant who provides answer to the user queries based only on the context and reason wherever necessary."},
        {"role": "user", "content": prompt}
    ]


    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=500,
        temperature=0.2,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=["\n"]
    )


    answer = response['choices'][0]['message']['content'].strip()
    conversation_history.append({"question": question, "answer": answer})
    return answer

In [60]:
print(chatbot("which is the fast content delivery network?"))

Amazon CloudFront is the fast content delivery network (CDN) service mentioned in the provided text. It securely delivers data, videos, applications, and APIs to customers globally with low latency and high transfer speeds within a developer-friendly environment.


In [61]:
print(chatbot("which company executes property-finding portals? and how many employees are there? also state th elocation of the company"))

The company that executes property-finding portals is the immowelt Group (IWG). The immowelt Group has more than 500 employees and is headquartered in Nuremberg, Germany.


In [63]:

# Function to summarize the conversation
def summarize_conversation():

    messages = [{"role": "system", "content": "You are a helpful assistant, summarize the conversation."},
                {"role": "user", "content": "Here is our conversation:"}]
    messages.extend([{"role": "user", "content": f"Q: {entry['question']} A: {entry['answer']}"} for entry in conversation_history])


    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=150,
        temperature=0.5,
        top_p=1.0,
        stop=None
    )

    return response.choices[0].message['content'].strip()




summary = summarize_conversation()
print("Summary of Conversation:", summary)

Summary of Conversation: The conversation discussed Amazon CloudFront as a fast content delivery network service, mentioning its ability to securely deliver data globally with low latency. Additionally, it highlighted the immowelt Group (IWG) as the company executing property-finding portals, with over 500 employees and headquartered in Nuremberg, Germany.
